# Crashlytics

In [1]:
try:
    import pyspark
except ModuleNotFoundError:
    !pip3 install pyspark
    import pyspark
try:
    import pandas as pd
except ModuleNotFoundError:
    !pip3 install pandas
    import pandas as pd
    import csv
try:
    import matplotlib.pyplot as plt
except ModuleNotFoundError:
    !pip3 install matplotlib
    import matplotlib.pyplot as plt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.0 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 8.4 MB/s eta 0:00:00
  DEPRECATION: pyspark is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559
  Running setup.py install for pyspark ... done

[notice] A new release of pip is available: 23.0 -> 23.3.1
[notice] To update, run: pip3 install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 10.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.5/502.5 kB 11.6 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━

In [2]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, StringType, LongType, IntegerType, FloatType
import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler, IndexToString, PCA
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator

In [3]:
#May need to install java for this to work
ss=SparkSession.builder.master("local").appName("crashlytics").getOrCreate()

23/11/14 10:55:47 WARN Utils: Your hostname, Aaryans-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 172.16.34.171 instead (on interface en0)
23/11/14 10:55:47 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/14 10:55:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
raw_df = ss.read.csv("smallest_crash_data.csv", header=True, inferSchema=True)
train_data, validation_data, test_data = raw_df.randomSplit([.6, .2, .2], seed=42)

In [5]:
raw_df.printSchema()
raw_df.show(5)

root
 |-- ID: string (nullable = true)
 |-- Source: string (nullable = true)
 |-- Severity: integer (nullable = true)
 |-- Start_Time: timestamp (nullable = true)
 |-- End_Time: timestamp (nullable = true)
 |-- Start_Lat: double (nullable = true)
 |-- Start_Lng: double (nullable = true)
 |-- End_Lat: string (nullable = true)
 |-- End_Lng: string (nullable = true)
 |-- Distance(mi): double (nullable = true)
 |-- Description: string (nullable = true)
 |-- Street: string (nullable = true)
 |-- City: string (nullable = true)
 |-- County: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Zipcode: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Timezone: string (nullable = true)
 |-- Airport_Code: string (nullable = true)
 |-- Weather_Timestamp: timestamp (nullable = true)
 |-- Temperature(F): double (nullable = true)
 |-- Wind_Chill(F): double (nullable = true)
 |-- Humidity(%): double (nullable = true)
 |-- Pressure(in): double (nullable = true)
 |-- V

23/11/14 10:56:00 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+---+-------+--------+-------------------+-------------------+-----------------+------------------+-------+-------+------------+--------------------+--------------------+------------+----------+-----+----------+-------+----------+------------+-------------------+--------------+-------------+-----------+------------+--------------+--------------+---------------+-----------------+-----------------+-------+-----+--------+--------+--------+-------+-------+----------+-------+-----+---------------+--------------+------------+--------------+--------------+-----------------+---------------------+
| ID| Source|Severity|         Start_Time|           End_Time|        Start_Lat|         Start_Lng|End_Lat|End_Lng|Distance(mi)|         Description|              Street|        City|    County|State|   Zipcode|Country|  Timezone|Airport_Code|  Weather_Timestamp|Temperature(F)|Wind_Chill(F)|Humidity(%)|Pressure(in)|Visibility(mi)|Wind_Direction|Wind_Speed(mph)|Precipitation(in)|Weather_Condition|Ameni

In [6]:
train_data.printSchema()
train_data.show(5)

root
 |-- ID: string (nullable = true)
 |-- Source: string (nullable = true)
 |-- Severity: integer (nullable = true)
 |-- Start_Time: timestamp (nullable = true)
 |-- End_Time: timestamp (nullable = true)
 |-- Start_Lat: double (nullable = true)
 |-- Start_Lng: double (nullable = true)
 |-- End_Lat: string (nullable = true)
 |-- End_Lng: string (nullable = true)
 |-- Distance(mi): double (nullable = true)
 |-- Description: string (nullable = true)
 |-- Street: string (nullable = true)
 |-- City: string (nullable = true)
 |-- County: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Zipcode: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Timezone: string (nullable = true)
 |-- Airport_Code: string (nullable = true)
 |-- Weather_Timestamp: timestamp (nullable = true)
 |-- Temperature(F): double (nullable = true)
 |-- Wind_Chill(F): double (nullable = true)
 |-- Humidity(%): double (nullable = true)
 |-- Pressure(in): double (nullable = true)
 |-- V

## Imbalance in Data

Crash severity of 2 or 3 is much more common than any other severity ranking, and crash durations (duration of impact on traffic) are recorded as 30 minutes the majority of the time. This shows that the data is imbalanced, which we will have to consider when building our classifier.

In [7]:
severity_count = train_data.groupBy(F.col("Severity")).count()
severity_count.show()

time_count = train_data.withColumn("time_diff", (F.col("Start_Time").cast("long") - F.col("End_Time").cast("long"))/60).groupBy(F.col("time_diff")).count()
time_count.orderBy(F.col("count"), ascending=False).show()

+--------+-----+
|Severity|count|
+--------+-----+
|       1|    1|
|       3|   48|
|       2|   73|
+--------+-----+

+------------------+-----+
|         time_diff|count|
+------------------+-----+
|             -30.0|   97|
|             -45.0|    9|
|             -60.0|    2|
|            -826.0|    1|
|-74.61666666666666|    1|
|            -137.0|    1|
|             -92.0|    1|
|            -350.0|    1|
|            -227.0|    1|
|             -72.0|    1|
|            -186.0|    1|
|            -156.0|    1|
|-49.06666666666667|    1|
|             -98.0|    1|
|            -144.0|    1|
|            -173.0|    1|
|            -153.0|    1|
+------------------+-----+



## TODO: clean data

Need to get rid of strings in data, by turning them into integer categories, removing them, or doing something clever with some other model that understands text

## PCA and K-Means Clustering

This data has a large number of features, so we will perform PCA to better understand which features of our dataset affect crash severity.

In [8]:
column_names = train_data.columns
assembler = VectorAssembler(inputCols=column_names, outputCol="features")
assembled_data = assembler.transform(train_data)
assembled_data.show(5)

IllegalArgumentException: Data type string of column ID is not supported.
Data type string of column Source is not supported.
Data type timestamp of column Start_Time is not supported.
Data type timestamp of column End_Time is not supported.
Data type string of column End_Lat is not supported.
Data type string of column End_Lng is not supported.
Data type string of column Description is not supported.
Data type string of column Street is not supported.
Data type string of column City is not supported.
Data type string of column County is not supported.
Data type string of column State is not supported.
Data type string of column Zipcode is not supported.
Data type string of column Country is not supported.
Data type string of column Timezone is not supported.
Data type string of column Airport_Code is not supported.
Data type timestamp of column Weather_Timestamp is not supported.
Data type string of column Wind_Direction is not supported.
Data type string of column Weather_Condition i

## Decision Tree Classifier

## Classification with XGBoost